In [ ]:
%matplotlib

In [ ]:

import numpy as np
import os, sys
import h5py
import pandas as pd
import matplotlib.pyplot as plt

import readdata as rd

import datetime
sys.path.append('/Users/ajc/Desktop/ATS/ats-repo/ats/tools/utils/')

In [ ]:
outfile = '/Users/ajc/AllPostProcessData/BarrowTransect/NGEE/Temperature-SiteC/'

In [ ]:
infile = '/Users/ajc/Desktop/ATS/barrow-polygon-data/Vladimir-Data/Area_C/'
sim_num = 4
simulation = 'Crun%sR22A'%sim_num
infile_simulated11 = '/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/NGEE/SiteC/Crun%sR21/'%sim_num
infile_simulated12 = '/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/NGEE/SiteC/%s/'%simulation
save = False

start_year = 2013
if start_year == 2013:
    print start_year
    dataset= 'NGEE_BRW_C_2012-09-09_2013-09-30_Thermal_Transect_and_Borehole.csv'
elif start_year == 2014:
    dataset= 'NGEE_BRW_C_2013-10-01_2014-09-30_Thermal_Transect_and_Borehole.csv'

infile_observed = infile + dataset
data = pd.read_csv(infile_observed, skiprows=5)
#print data.columns
DataObservedTrough = dict()
DataObservedCenter = dict()
DataObservedRim = dict()

#origin=pd.Timestamp('1960-01-01')
d = dict()
time_origin = datetime.datetime(start_year,1,1,0,0,0)
d['time'] = [pd.to_datetime(data['Timestamp'])]


Time = [(t - time_origin).dt.total_seconds()/86400 for t in d[u'time']]
Time = np.array([t for t in Time[0]])
#print Time[0]
start_time = np.where(Time == 0)[0]
print start_time
start_time = start_time[0]

for c in data.columns:
    if 'trough' in c:
        k = int(float(c[6:10])*100)
        d = np.array([x + 273.15  for x in  data[c]])
        DataObservedTrough['temperature-%scm'%k] = d
    elif 'center' in c and not 'off' in c:
        k = int(float(c[6:10])*100)
        #print k, c
        d = np.array([ x + 273.15  for x in data[c]])
        #d = np.array([  d1[0] if np.isnan(x) else x  for i,x in enumerate(d1)])
        
        DataObservedCenter['temperature-%scm'%k] = d
    elif 'rim' in c and 'rim2' not in c:
        k = int(float(c[6:10])*100)
        d = np.array([x + 273.15  for x in data[c]])
        if k != 2 and start_year == 2014:
            k = k + 4
        #print k, c
        DataObservedRim['temperature-%scm'%k] = d
DataObservedCenter['time'] = Time  
DataObservedTrough['time'] = Time  
DataObservedRim['time'] = Time  
for key, d in DataObservedCenter.iteritems():
    DataObservedCenter[key] = np.array(DataObservedCenter[key])
for key, d in DataObservedTrough.iteritems():
    DataObservedTrough[key] = np.array(DataObservedTrough[key])
for key, d in DataObservedRim.iteritems():
    DataObservedRim[key] = np.array(DataObservedRim[key])

In [ ]:

depth = ['2cm','5cm', '10cm', '15cm', '20cm', '25cm', '30cm', '35cm', '40cm', '50cm', '60cm', '70cm', '80cm', '100cm', '125cm', '150cm']


DataSim11Trough = dict()
DataSim11Center = dict()
DataSim11Rim = dict()
TimeSim11 = []
for d in depth:
    file = infile_simulated11 + 'temperature-center-%s.dat'%d
    data = rd.ReadSingleFile(file)
    DataSim11Center['temperature-%s'%d] = data['data']

    file = infile_simulated11 + 'temperature-trough-%s.dat'%d
    data = rd.ReadSingleFile(file)
    #print data
    #break
    DataSim11Trough['temperature-%s'%d] = data['data']
    
    file = infile_simulated11 + 'temperature-rim-%s.dat'%d
    data = rd.ReadSingleFile(file)
    DataSim11Rim['temperature-%s'%d] = data['data']
    
TimeSim11= [t/86400. for t in data['time']]

TimeSim11 = np.array(TimeSim11)

#print DataSimTrough['temperature-100cm'][:2]

In [ ]:

depth = ['2cm','5cm', '10cm', '15cm', '20cm', '25cm', '30cm', '35cm', 
         '40cm', '50cm', '60cm', '70cm', '80cm', '100cm', '125cm', '150cm']

DataSim12Trough = dict()
DataSim12Center = dict()
DataSim12Rim = dict()
TimeSim12 = []
for d in depth:
    file = infile_simulated12 + 'temperature-center-%s.dat'%d
    data = rd.ReadSingleFile(file)
    DataSim12Center['temperature-%s'%d] = data['data']

    file = infile_simulated12 + 'temperature-trough-%s.dat'%d
    data = rd.ReadSingleFile(file)
    DataSim12Trough['temperature-%s'%d] = data['data']
    
    file = infile_simulated12 + 'temperature-rim-%s.dat'%d
    data = rd.ReadSingleFile(file)
    DataSim12Rim['temperature-%s'%d] = data['data']
    
TimeSim12 = [t/86400. for t in data['time']]

TimeSim12 = np.array(TimeSim12)


In [ ]:
def put_axis(dep, loc=''):
    plt.xlabel('Day (%s)'%start_year,fontsize=12, fontweight='normal')
    plt.ylabel('Temperature [K]',fontsize=12, fontweight='normal')
    #plt.text(4, 285, '-%s, %s, %s'%(dep,start_year, loc),fontsize=12, fontweight='normal')
    plt.text(4, 282, 'depth = %s'%dep,fontsize=12, fontweight='normal')
    plt.text(4, 278, '%s'%(loc),fontsize=12, fontweight='normal')
    plt.ylim((245, 285))
    plt.yticks(np.linspace(245,285, 5))
    plt.xlim((0, 280))
    plt.xticks(np.linspace(0,280, 5))
    plt.axhline(y=273.25,color='b')

DataObservedCenterDaily = dict()
DataObservedTroughDaily = dict()
DataObservedRimDaily = dict()
DataSim11TroughDaily = dict()
DataSim11CenterDaily = dict()
DataSim11RimDaily = dict()
DataSim12TroughDaily = dict()
DataSim12CenterDaily = dict()
DataSim12RimDaily = dict()

l = len(DataObservedCenter['temperature-2cm'])
r = l%24


for key, d in DataObservedCenter.iteritems():
    DataObservedCenterDaily[key] = np.reshape(DataObservedCenter[key][:-r], (-1, 24)).mean(axis=1)
for key, d in DataObservedTrough.iteritems():
    DataObservedTroughDaily[key] = np.reshape(DataObservedTrough[key][:-r], (-1, 24)).mean(axis=1)
for key, d in DataObservedRim.iteritems():
    DataObservedRimDaily[key] = np.reshape(DataObservedRim[key][:-r], (-1, 24)).mean(axis=1)

l = len(DataSim11Center['temperature-2cm'])
r = l%24
for key, d in DataSim11Center.iteritems():
    DataSim11CenterDaily[key] = np.reshape(DataSim11Center[key][:-r], (-1, 24)).mean(axis=1)
for key, d in DataSim11Trough.iteritems():
    DataSim11TroughDaily[key] = np.reshape(DataSim11Trough[key][:-r], (-1, 24)).mean(axis=1)
for key, d in DataSim11Rim.iteritems():
    DataSim11RimDaily[key] = np.reshape(DataSim11Rim[key][:-r], (-1, 24)).mean(axis=1)
    
TimeSim11Daily = np.reshape(TimeSim11[:-r], (-1,24)).mean(axis=1)

l = len(DataSim12Center['temperature-2cm'])
r = l%24
for key, d in DataSim12Center.iteritems():
    DataSim12CenterDaily[key] = np.reshape(DataSim12Center[key][:-r], (-1, 24)).mean(axis=1)
for key, d in DataSim12Trough.iteritems():
    DataSim12TroughDaily[key] = np.reshape(DataSim12Trough[key][:-r], (-1, 24)).mean(axis=1)
for key, d in DataSim12Rim.iteritems():
    DataSim12RimDaily[key] = np.reshape(DataSim12Rim[key][:-r], (-1, 24)).mean(axis=1)
    
TimeSim12Daily = np.reshape(TimeSim12[:-r], (-1,24)).mean(axis=1)

In [ ]:
# Polygon Center
fig, axs = plt.subplots(3,3, figsize=(8,8), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1,1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)

plt.tight_layout()
axs = axs.ravel()

plt.subplot(331)
#plt.plot(Time[start_time:], DataObservedCenter['temperature-5cm'][start_time:],'r')
plt.plot(DataObservedCenterDaily['time'], DataObservedCenterDaily['temperature-5cm'],'r-')
plt.plot(TimeSim11Daily, DataSim11CenterDaily['temperature-5cm'],'k')
plt.plot(TimeSim12Daily, DataSim12CenterDaily['temperature-5cm'],'k')
put_axis('5cm', 'center')

plt.subplot(332)
#plt.plot(Time[start_time:], DataObservedCenter['temperature-10cm'][start_time:],'r')
plt.plot(DataObservedCenterDaily['time'], DataObservedCenterDaily['temperature-10cm'],'r-')
plt.plot(TimeSim11Daily, DataSim11CenterDaily['temperature-10cm'],'k')
plt.plot(TimeSim12Daily, DataSim12CenterDaily['temperature-10cm'],'k')
put_axis('10cm', 'center')

plt.subplot(333)
#plt.plot(Time[start_time:], DataObservedCenter['temperature-20cm'][start_time:],'r')
plt.plot(DataObservedCenterDaily['time'], DataObservedCenterDaily['temperature-20cm'],'r-')
plt.plot(TimeSim11Daily, DataSim11CenterDaily['temperature-20cm'],'k')
plt.plot(TimeSim12Daily, DataSim12CenterDaily['temperature-20cm'],'k')
put_axis('20cm', 'center')

plt.subplot(334)
#plt.plot(Time[start_time:], DataObservedCenter['temperature-40cm'][start_time:],'r')
plt.plot(DataObservedCenterDaily['time'], DataObservedCenterDaily['temperature-40cm'],'r-')
plt.plot(TimeSim11Daily, DataSim11CenterDaily['temperature-40cm'],'k')
plt.plot(TimeSim12Daily, DataSim12CenterDaily['temperature-40cm'],'k')
put_axis('40cm', 'center')

plt.subplot(335)
#plt.plot(Time[start_time:], DataObservedCenter['temperature-60cm'][start_time:],'r')
plt.plot(DataObservedCenterDaily['time'], DataObservedCenterDaily['temperature-60cm'],'r-')
plt.plot(TimeSim11Daily, DataSim11CenterDaily['temperature-60cm'],'k')
plt.plot(TimeSim12Daily, DataSim12CenterDaily['temperature-60cm'],'k')
put_axis('60cm', 'center')

plt.subplot(336)
#plt.plot(Time[start_time:], DataObservedCenter['temperature-80cm'][start_time:],'r')
plt.plot(DataObservedCenterDaily['time'], DataObservedCenterDaily['temperature-80cm'],'r-')
plt.plot(TimeSim11Daily, DataSim11CenterDaily['temperature-80cm'],'k')
plt.plot(TimeSim12Daily, DataSim12CenterDaily['temperature-80cm'],'k')
put_axis('80cm', 'center')

plt.subplot(337)
#plt.plot(Time[start_time:], DataObservedCenter['temperature-100cm'][start_time:],'r')
plt.plot(DataObservedCenterDaily['time'], DataObservedCenterDaily['temperature-100cm'],'r-')
plt.plot(TimeSim11Daily, DataSim11CenterDaily['temperature-100cm'],'k')
plt.plot(TimeSim12Daily, DataSim12CenterDaily['temperature-100cm'],'k')
put_axis('100cm', 'center')

plt.subplot(338)
#plt.plot(Time[start_time:], DataObservedCenter['temperature-125cm'][start_time:],'r')
plt.plot(DataObservedCenterDaily['time'], DataObservedCenterDaily['temperature-125cm'],'r-')
plt.plot(TimeSim11Daily, DataSim11CenterDaily['temperature-125cm'],'k')
plt.plot(TimeSim12Daily, DataSim12CenterDaily['temperature-125cm'],'k')
put_axis('125cm', 'center')

plt.subplot(339)
#plt.plot(Time[start_time:], DataObservedCenter['temperature-150cm'][start_time:],'r',label='Observed (hourly)')
plt.plot(DataObservedCenterDaily['time'], DataObservedCenterDaily['temperature-150cm'],'r',label='Observed')
plt.plot(TimeSim11Daily, DataSim11CenterDaily['temperature-150cm'],'k',label='Simulated')
plt.plot(TimeSim12Daily, DataSim12CenterDaily['temperature-150cm'],'k')
put_axis('150cm', 'center')

plt.legend(loc='upper right', fontsize=10, ncol=3, bbox_to_anchor=(0.7,- 0.32, .1, .1))

plt.tight_layout()
if save == True:
    plt.savefig(outfile+'temp-center%s-%s-daily--.pdf'%(start_year,simulation), bbox_inches='tight', dpi=600)

In [ ]:
# Polygon Trough
fig, axs = plt.subplots(3,3, figsize=(8,8), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1,1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)

plt.tight_layout()
axs = axs.ravel()

plt.subplot(331)
#plt.plot(Time[start_time:], DataObservedTroughDaily['temperature-5cm'][start_time:],'r')
plt.plot(DataObservedTroughDaily['time'], DataObservedTroughDaily['temperature-5cm'],'r-')
plt.plot(TimeSim11Daily, DataSim11TroughDaily['temperature-5cm'],'k')
plt.plot(TimeSim12Daily, DataSim12TroughDaily['temperature-5cm'],'k')
put_axis('5cm', 'Trough')

plt.subplot(332)
#plt.plot(Time[start_time:], DataObservedTroughDaily['temperature-10cm'][start_time:],'r')
plt.plot(DataObservedTroughDaily['time'], DataObservedTroughDaily['temperature-10cm'],'r-')
plt.plot(TimeSim11Daily, DataSim11TroughDaily['temperature-10cm'],'k')
plt.plot(TimeSim12Daily, DataSim12TroughDaily['temperature-10cm'],'k')
put_axis('10cm', 'Trough')

plt.subplot(333)
#plt.plot(Time[start_time:], DataObservedTroughDaily['temperature-20cm'][start_time:],'r')
plt.plot(DataObservedTroughDaily['time'], DataObservedTroughDaily['temperature-20cm'],'r-')
plt.plot(TimeSim11Daily, DataSim11TroughDaily['temperature-20cm'],'k')
plt.plot(TimeSim12Daily, DataSim12TroughDaily['temperature-20cm'],'k')
put_axis('20cm', 'Trough')

plt.subplot(334)
#plt.plot(Time[start_time:], DataObservedTroughDaily['temperature-40cm'][start_time:],'r')
plt.plot(DataObservedTroughDaily['time'], DataObservedTroughDaily['temperature-40cm'],'r-')
plt.plot(TimeSim11Daily, DataSim11TroughDaily['temperature-40cm'],'k')
plt.plot(TimeSim12Daily, DataSim12TroughDaily['temperature-40cm'],'k')
put_axis('40cm', 'Trough')

plt.subplot(335)
#plt.plot(Time[start_time:], DataObservedTroughDaily['temperature-60cm'][start_time:],'r')
plt.plot(DataObservedTroughDaily['time'], DataObservedTroughDaily['temperature-60cm'],'r-')
plt.plot(TimeSim11Daily, DataSim11TroughDaily['temperature-60cm'],'k')
plt.plot(TimeSim12Daily, DataSim12TroughDaily['temperature-60cm'],'k')
put_axis('60cm', 'Trough')

plt.subplot(336)
#plt.plot(Time[start_time:], DataObservedTroughDaily['temperature-80cm'][start_time:],'r')
plt.plot(DataObservedTroughDaily['time'], DataObservedTroughDaily['temperature-80cm'],'r-')
plt.plot(TimeSim11Daily, DataSim11TroughDaily['temperature-80cm'],'k')
plt.plot(TimeSim12Daily, DataSim12TroughDaily['temperature-80cm'],'k')
put_axis('80cm', 'Trough')

plt.subplot(337)
#plt.plot(Time[start_time:], DataObservedTroughDaily['temperature-100cm'][start_time:],'r')
plt.plot(DataObservedTroughDaily['time'], DataObservedTroughDaily['temperature-100cm'],'r-')
plt.plot(TimeSim11Daily, DataSim11TroughDaily['temperature-100cm'],'k')
plt.plot(TimeSim12Daily, DataSim12TroughDaily['temperature-100cm'],'k')
put_axis('100cm', 'Trough')

plt.subplot(338)
#plt.plot(Time[start_time:], DataObservedTroughDaily['temperature-125cm'][start_time:],'r')
plt.plot(DataObservedTroughDaily['time'], DataObservedTroughDaily['temperature-125cm'],'r-')
plt.plot(TimeSim11Daily, DataSim11TroughDaily['temperature-125cm'],'k')
plt.plot(TimeSim12Daily, DataSim12TroughDaily['temperature-125cm'],'k')
put_axis('125cm', 'Trough')

plt.subplot(339)
#plt.plot(Time[start_time:], DataObservedTroughDaily['temperature-150cm'][start_time:],'r',label='Observed (hourly)')
plt.plot(DataObservedTroughDaily['time'], DataObservedTroughDaily['temperature-150cm'],'r-',label='Observed')
plt.plot(TimeSim11Daily, DataSim11TroughDaily['temperature-150cm'],'k',label='Simulated')
plt.plot(TimeSim12Daily, DataSim12TroughDaily['temperature-150cm'],'k')
put_axis('150cm', 'Trough')

plt.legend(loc='upper right', fontsize=10, ncol=3, bbox_to_anchor=(0.7,- 0.32, .1, .1))

plt.tight_layout()
if save == True:
    plt.savefig(outfile+'temp-Trough%s-%s-daily.pdf'%(start_year,simulation), bbox_inches='tight', dpi=600)

In [ ]:
# Polygon Rim
fig, axs = plt.subplots(3,3, figsize=(8,8), facecolor='w', edgecolor='k',gridspec_kw={'width_ratios':[1,1,1]})
fig.subplots_adjust(hspace =.02, wspace=1.02)

plt.tight_layout()
axs = axs.ravel()

plt.subplot(331)
#plt.plot(Time[start_time:], DataObservedRimDaily['temperature-5cm'][start_time:],'r')
plt.plot(DataObservedRimDaily['time'], DataObservedRimDaily['temperature-5cm'],'r-')
plt.plot(TimeSim11Daily, DataSim11RimDaily['temperature-5cm'],'k')
plt.plot(TimeSim12Daily, DataSim12RimDaily['temperature-5cm'],'k')
put_axis('5cm', 'Rim')

plt.subplot(332)
#plt.plot(Time[start_time:], DataObservedRimDaily['temperature-10cm'][start_time:],'r')
plt.plot(DataObservedRimDaily['time'], DataObservedRimDaily['temperature-10cm'],'r-')
plt.plot(TimeSim11Daily, DataSim11RimDaily['temperature-10cm'],'k')
plt.plot(TimeSim12Daily, DataSim12RimDaily['temperature-10cm'],'k')
put_axis('10cm', 'Rim')

plt.subplot(333)
#plt.plot(Time[start_time:], DataObservedRimDaily['temperature-20cm'][start_time:],'r')
plt.plot(DataObservedRimDaily['time'], DataObservedRimDaily['temperature-20cm'],'r-')
plt.plot(TimeSim11Daily, DataSim11RimDaily['temperature-20cm'],'k')
plt.plot(TimeSim12Daily, DataSim12RimDaily['temperature-20cm'],'k')
put_axis('20cm', 'Rim')

plt.subplot(334)
#plt.plot(Time[start_time:], DataObservedRimDaily['temperature-40cm'][start_time:],'r')
plt.plot(DataObservedRimDaily['time'], DataObservedRimDaily['temperature-40cm'],'r-')
plt.plot(TimeSim11Daily, DataSim11RimDaily['temperature-40cm'],'k')
plt.plot(TimeSim12Daily, DataSim12RimDaily['temperature-40cm'],'k')
put_axis('40cm', 'Rim')

plt.subplot(335)
#plt.plot(Time[start_time:], DataObservedRimDaily['temperature-60cm'][start_time:],'r')
plt.plot(DataObservedRimDaily['time'], DataObservedRimDaily['temperature-60cm'],'r-')
plt.plot(TimeSim11Daily, DataSim11RimDaily['temperature-60cm'],'k')
plt.plot(TimeSim12Daily, DataSim12RimDaily['temperature-60cm'],'k')
put_axis('60cm', 'Rim')

plt.subplot(336)
#plt.plot(Time[start_time:], DataObservedRimDaily['temperature-80cm'][start_time:],'r')
plt.plot(DataObservedRimDaily['time'], DataObservedRimDaily['temperature-80cm'],'r-')
plt.plot(TimeSim11Daily, DataSim11RimDaily['temperature-80cm'],'k')
plt.plot(TimeSim12Daily, DataSim12RimDaily['temperature-80cm'],'k')
put_axis('80cm', 'Rim')

plt.subplot(337)
#plt.plot(Time[start_time:], DataObservedRimDaily['temperature-100cm'][start_time:],'r')
plt.plot(DataObservedRimDaily['time'], DataObservedRimDaily['temperature-100cm'],'r-')
plt.plot(TimeSim11Daily, DataSim11RimDaily['temperature-100cm'],'k')
plt.plot(TimeSim12Daily, DataSim12RimDaily['temperature-100cm'],'k')
put_axis('100cm', 'Rim')

plt.subplot(338)
#plt.plot(Time[start_time:], DataObservedRimDaily['temperature-125cm'][start_time:],'r')
plt.plot(DataObservedRimDaily['time'], DataObservedRimDaily['temperature-125cm'],'r-')
plt.plot(TimeSim11Daily, DataSim11RimDaily['temperature-125cm'],'k')
plt.plot(TimeSim12Daily, DataSim12RimDaily['temperature-125cm'],'k')
put_axis('125cm', 'Rim')

plt.subplot(339)
#plt.plot(Time[start_time:], DataObservedRimDaily['temperature-150cm'][start_time:],'r',label='Observed (hourly)')
plt.plot(DataObservedRimDaily['time'], DataObservedRimDaily['temperature-150cm'],'r-', label='Observed')
plt.plot(TimeSim11Daily, DataSim11RimDaily['temperature-150cm'],'k', label='Simulated')
plt.plot(TimeSim12Daily, DataSim12RimDaily['temperature-150cm'],'k')
put_axis('150cm', 'Rim')

plt.legend(loc='upper right', fontsize=10, ncol=3, bbox_to_anchor=(0.7,- 0.32, .1, .1))

plt.tight_layout()
if save == True:
    plt.savefig(outfile+'temp-Rim%s-%s-daily.pdf'%(start_year,simulation), bbox_inches='tight', dpi=600)

In [ ]:
plt.plot(DataObservedCenterDaily['time'], DataObservedCenterDaily['temperature-10cm'],'r-', label='Observed (center)')

#put_axis('40cm', 'Center')

plt.plot(DataObservedRimDaily['time'], DataObservedRimDaily['temperature-10cm'],'k', label='Observed (rim)')

#put_axis('40cm', 'Rim')

plt.plot(DataObservedTroughDaily['time'], DataObservedTroughDaily['temperature-10cm'],'g-', label='Observed (trough)')

put_axis('10cm')
plt.legend(loc = 'lower right')
plt.show()


In [ ]:
#plt.close()

plt.plot(TimeSim11Daily, DataSim11CenterDaily['temperature-10cm'],'r', label='Simulated (center)')
plt.plot(TimeSim12Daily, DataSim12CenterDaily['temperature-10cm'],'r')

plt.plot(TimeSim11Daily, DataSim11RimDaily['temperature-10cm'],'k', label='Simulated (rim)')
plt.plot(TimeSim12Daily, DataSim12RimDaily['temperature-10cm'],'k')

plt.plot(TimeSim11, DataSim11Trough['temperature-10cm'],'g', label='Simulated (trough)')
plt.plot(TimeSim12, DataSim12Trough['temperature-10cm'],'g')

put_axis('10cm')
plt.legend(loc = 'lower right')

#plt.show()

In [ ]:
import plot_water_table2D
times, wt, wt_ss, wt_bottom,thaw_depth = plot_water_table2D.water_table('/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/polygon-slice/AreaC/%s/'%simulation)

In [ ]:
lower_point = 4.8284937 
lower_point=  4.80342948 # new mesh
lower_point = 4.81031227 # new smooth mesh

In [ ]:
cellid = 36
cellid = 44 # new mesh
metdata = rd.GetSurfVarFromVis('/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/polygon-slice/AreaC/%s/visdump_surface_data.h5'%simulation, 'surface-precipitation_rain', cellid=cellid)
surftemp = rd.GetSurfVarFromVis('/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/polygon-slice/AreaC/%s/visdump_surface_data.h5'%simulation, 'surface-temperature', cellid=cellid)
#plt.plot(metdata['surface-precipitation_rain'])
#plt.show()

In [ ]:
rain = [d *100*900./0.02500 for d in metdata['surface-precipitation_rain']]
E1 = 4.72  + 0.075 * np.exp(rain)
#plt.plot(Data37['time'], Data37['data'], 'k', label='Simulated')
#plt.plot(DataC37['time'], DataC37['WL'],'r', label='Observed (C37)')

#plt.plot(Data37['time'],E1,'*')
#plt.show()
#simulation1 = 'CrunB30R79'
#times1, wt1, wt_ss1, wt_bottom1,thaw_depth1 = plot_water_table2D.water_table('/Users/ajc/Projects/ATS-Data/OR-CONDO/mytests-orhydra/Barrow/polygon-slice/AreaC/%s/'%simulation1)

In [ ]:
Data40 =dict()
time = [t*365.25 for t in times]
#time = [t for t in times]
Data40['time'] = time
Data40['wt'] = wt
Data40['wt_ss'] = wt_ss
Data40['wt_bottom'] = wt_bottom
print (times[0:3])
        
D1 = []
T1 = []
for i,d in enumerate(Data40['wt']):
    if d != 0:
        D1.append(d + lower_point)
        T1.append(Data40['time'][i])
        
D2 = []
T2 = []
for i,d in enumerate(Data40['wt']):
    D2.append(d + lower_point)
    T2.append(Data40['time'][i])
    
#water table bottom
D3 = []
T3 = []
for i,d in enumerate(Data40['wt_bottom']):
    D3.append(d + lower_point)
    T3.append(Data40['time'][i])
TD = []
for d in thaw_depth:
    TD.append(d + lower_point)   

"""
# Second simulation
Data401 =dict()
time1 = [t*365.25 for t in times]
Data401['time'] = time1
Data401['wt'] = wt1
Data401['wt_ss'] = wt_ss1
Data401['wt_bottom'] = wt_bottom1
        
D11 = []
T11 = []
for i,d in enumerate(Data401['wt']):
    if d != 0:
        D11.append(d + lower_point)
        T11.append(Data401['time'][i])
        
    
#water table bottom
D31 = []
T31 = []
for i,d in enumerate(Data401['wt_bottom']):
    D31.append(d + lower_point)
    T31.append(Data401['time'][i])
TD1 = []
for d in thaw_depth1:
    TD1.append(d + lower_point) 
"""